```
dx download Annotation/gene/gencode.v35.annotation.gff3.gz
dx download chr-subset-genes-variants/all_variants_chr21.tsv
```

In [ ]:
library(rtracklayer)

In [ ]:
genc = import('gencode.v35.annotation.gff3.gz')
genc

In [3]:
genc

GRanges object with 2949356 ranges and 23 metadata columns:
            seqnames      ranges strand |   source       type     score
               <Rle>   <IRanges>  <Rle> | <factor>   <factor> <numeric>
        [1]     chr1 11869-14409      + |   HAVANA       gene      <NA>
        [2]     chr1 11869-14409      + |   HAVANA transcript      <NA>
        [3]     chr1 11869-12227      + |   HAVANA       exon      <NA>
        [4]     chr1 12613-12721      + |   HAVANA       exon      <NA>
        [5]     chr1 13221-14409      + |   HAVANA       exon      <NA>
        ...      ...         ...    ... .      ...        ...       ...
  [2949352]     chrM 15888-15953      + |  ENSEMBL transcript      <NA>
  [2949353]     chrM 15888-15953      + |  ENSEMBL       exon      <NA>
  [2949354]     chrM 15956-16023      - |  ENSEMBL       gene      <NA>
  [2949355]     chrM 15956-16023      - |  ENSEMBL transcript      <NA>
  [2949356]     chrM 15956-16023      - |  ENSEMBL       exon      <NA>
    

In [4]:
table(genc$type)


                                  gene                             transcript 
                                 60656                                 229580 
                                  exon                                    CDS 
                               1398443                                 774993 
                           start_codon                             stop_codon 
                                 88717                                  81122 
                        five_prime_UTR                        three_prime_UTR 
                                155436                                 160294 
stop_codon_redefined_as_selenocysteine 
                                   115 

In [5]:
genc = subset(genc, type %in% c('transcript', 'CDS', 'five_prime_UTR', 'three_prime_UTR'))

In [6]:
library(GenomicRanges)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
vars = read.table('all_variants_chr21.tsv', as.is=TRUE, sep='\t', header=TRUE)
vars.gr = vars %>% mutate(chr=paste0('chr', chr)) %>% makeGRangesFromDataFrame(keep.extra.columns = TRUE)
vars.gr

GRanges object with 420225 ranges and 2 metadata columns:
           seqnames          ranges strand |        type   variant_id
              <Rle>       <IRanges>  <Rle> | <character>  <character>
       [1]    chr21 5051593-5051594      * |         INS nssv14017801
       [2]    chr21 5064052-5066138      * |         DEL nssv14300595
       [3]    chr21 5064052-5066138      * |         DEL nssv14301211
       [4]    chr21 5064052-5066138      * |         DEL nssv14301212
       [5]    chr21 5064052-5066138      * |         DEL nssv14301213
       ...      ...             ...    ... .         ...          ...
  [420221]    chr21        46699865      * |         INS nssv14662319
  [420222]    chr21        46699867      * |         INS nssv14667619
  [420223]    chr21        46699891      * |         INS nssv14670897
  [420224]    chr21        46699928      * |         INS nssv14657123
  [420225]    chr21        46699957      * |         INS nssv14663905
  -------
  seqinfo: 1 sequence 

In [9]:
var.gene.df = findOverlaps(vars.gr, genc) %>% as.data.frame %>%
    filter(genc$gene_type[subjectHits] == 'protein_coding') %>%
    mutate(variant_id=vars.gr$variant_id[queryHits],
           gene_id=genc$gene_id[subjectHits],
           gene_name=genc$gene_name[subjectHits],
           transcript_id=genc$transcript_id[subjectHits],
           exon_number=genc$exon_number[subjectHits], type=genc$type[subjectHits]) %>% 
    select(-queryHits, -subjectHits) %>% unique
head(var.gene.df)
nrow(var.gene.df)

,variant_id,gene_id,gene_name,transcript_id,exon_number,type
,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>
1,nssv13693287,ENSG00000280071.4,GATD3B,ENST00000624810.3,NA,transcript
2,nssv13693287,ENSG00000280071.4,GATD3B,ENST00000625036.3,NA,transcript
3,nssv14395136,ENSG00000280071.4,GATD3B,ENST00000624810.3,NA,transcript
4,nssv14395136,ENSG00000280071.4,GATD3B,ENST00000625036.3,NA,transcript
5,nssv14031467,ENSG00000280071.4,GATD3B,ENST00000624810.3,NA,transcript
6,nssv14031467,ENSG00000280071.4,GATD3B,ENST00000625036.3,NA,transcript


[1] 9448086

In [10]:
var.gene.df = var.gene.df %>% group_by(variant_id, gene_id, gene_name, transcript_id, type) %>% summarize(exon_number=paste(sort(unique(exon_number)), collapse=';'))
head(var.gene.df)
nrow(var.gene.df)

`summarise()` regrouping output by 'variant_id', 'gene_id', 'gene_name', 'transcript_id' (override with `.groups` argument)



variant_id,gene_id,gene_name,transcript_id,type,exon_number
<chr>,<chr>,<chr>,<chr>,<fct>,<chr>
essv100139,ENSG00000160209.19,PDXK,ENST00000291565.9,transcript,
essv100139,ENSG00000160209.19,PDXK,ENST00000291565.9,CDS,3
essv100139,ENSG00000160209.19,PDXK,ENST00000327574.4,transcript,
essv100139,ENSG00000160209.19,PDXK,ENST00000327574.4,three_prime_UTR,4
essv100139,ENSG00000160209.19,PDXK,ENST00000343528.10,transcript,
essv100139,ENSG00000160209.19,PDXK,ENST00000398078.7,transcript,


[1] 3582430

In [11]:
write.table(var.gene.df, file='variants.genes.chr21.tsv', sep='\t', quote=FALSE, row.names=FALSE)

```
gzip variants.genes.chr21.tsv
dx upload variants.genes.chr21.tsv.gz --path jmonlong-notebook-test/
dx upload variant-gene-overlap.ipynb --path jmonlong-notebook-test/
```